<a href="https://colab.research.google.com/github/alicetw40342/Building-a-PDF-Based-Question-Answering-System-with-RAG-and-Gemini/blob/main/Project3_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ 安裝必要套件
!pip install -q sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.7 MB/s eta 

In [ ]:
# ✅ 匯入套件與初始化模型
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
import chromadb

# 使用輕量快速的本地模型
local_model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
# ✅ 定義本地嵌入工具與距離計算
class EmbeddingUtils:
    @staticmethod
    def get_embedding(text: str, model="local") -> List[float]:
        return local_model.encode(text).tolist()

    @staticmethod
    def distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine") -> List[float]:
        similarities = cosine_similarity([query_embedding], embeddings)[0]
        distances = [1 - sim for sim in similarities]
        return distances

    @staticmethod
    def indices_of_nearest_neighbors_from_distances(distances: List[float]) -> List[int]:
        return sorted(range(len(distances)), key=lambda i: distances[i])


In [ ]:
# ✅ 建立 Chroma 向量資料庫並填入嵌入資料
def create_chroma_db(documents: List[str], path: str, name: str):
    """
    使用本地模型取得嵌入，並存入 Chroma 向量資料庫
    """
    client = chromadb.PersistentClient(path=path)
    collection = client.get_or_create_collection(name=name)

    utils = EmbeddingUtils()
    embeddings = [utils.get_embedding(doc) for doc in documents]
    ids = [f"doc_{i}" for i in range(len(documents))]

    collection.add(
        documents=documents,
        embeddings=embeddings,
        ids=ids
    )

    return collection


In [ ]:
# ✅ 測試資料與執行結果
test_chunks = [
    "Artificial intelligence is transforming the world.",
    "Machine learning is a technique in AI.",
    "Neural networks mimic the human brain."
]

db_path = "/content/chroma_db"
collection_name = "pdf_chunks"

collection = create_chroma_db(test_chunks, db_path, collection_name)
print("✅ 向量資料庫建立成功，共儲存筆數：", collection.count())
